Oceanography python bootcamp, Winter 2025
# Week 7 notebook

In [ ]:
import os

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

import scipy.optimize as sopt
import scipy.integrate as sint

## Nonlinear equation solving

### 1D example: $\tan(x) = x$

In [ ]:
# first define the function we want the root for
def f(x):
    return np.tan(x) - x

In [ ]:
# next find a suitable bracketing interval
bracket = np.array([np.pi, 3*np.pi/2 - 0.1])

print(bracket)
print(f(bracket))

In [ ]:
# call root_scalar() to find root
# NOTE: the default method of root_scalar() is brentq
soln = sopt.root_scalar(f, bracket=bracket)

In [ ]:
# print for information
print(soln)

In [ ]:
# extract the root programatically
soln.root

### Simultaneous equations example: a pair of polynomial equations

Suppose we're to simultaneously solve the pair of polynomial equations in the first quadrant:
$$ x^2 + y^2 = 10 $$
$$ x^2 y - x y^3 - xy = -3 $$

where $x$ and $y$ are non-negative.

In [ ]:
# first we define the equation to solve
def g(x):
    y1 = x[0]**2 + x[1]**2 - 10 # first equation
    y2 = x[0] * x[1] * (x[0] - x[1]**2 - 1) + 3 # second equation
    return np.array([y1, y2])

In [ ]:
# Visualize the situation using contour
x = np.linspace(0, 5, 51)
y = np.linspace(0, 5, 51)

x_grid, y_grid = np.meshgrid(x, y)

z = g(np.array([x_grid, y_grid]))

print(x_grid.shape, y_grid.shape, z.shape)

fig = plt.figure()
ax = fig.add_subplot()

ax.set_aspect(1)

c1 = ax.contour(x, y, z[0, :, :], colors="red", linestyles="--")
c2 = ax.contour(x, y, z[1, :, :], colors="blue", linestyles=":")

c1.clabel()
c2.clabel()

handles1, _ = c1.legend_elements()
handles2, _ = c2.legend_elements()

ax.legend(
    handles = handles1[:1] + handles2[:1], 
    labels = ["$ x^2 + y^2 - 10 $", "$ x^2 y - x y^3 - xy + 3 $"], 
    loc="lower right"
)

plt.show()

In [ ]:
# try to find the first root
soln1 = sopt.root(g, np.array([3, 1]))

In [ ]:
print(soln1)

In [ ]:
# try to find the second root
soln2 = sopt.root(g, np.array([0, 3]))

In [ ]:
print(soln2)

In [ ]:
# extract the root programmatically:
soln1.x

In [ ]:
soln2.x

### Exercise 1

Water flows in an open, rectangular channel of constant width (w). There is a small ramp (height δ) downstream. From the Bernoulli equation and the continuity equation, we obtain:

$$ \frac{Q^2}{2 g w^2} \left(\frac{1}{h_0^2} - \frac{1}{h^2}\right) - (h - h_0) - δ = 0 $$

where $g$ = 9.8 m/s$^2$ is the acceleration of gravity, h0 and h are the water levels, and $Q$ is the volume flow rate. For $Q$ = 0.4 m$^3$/s, $w$ = 0.8 m, $h_0$ = 0.5 m and $δ$ = 0.07 m, estimate the elevation of the water surface $h + δ$ downstream of the ramp


## Numerical integrations

### Example: $\int_0^4 x^2 dx$

In [ ]:
def f1(x):
    return x**2

integral1 = sint.quad(f1, 0, 4)
print(integral1)

In [ ]:
# compare to exact answer
exact1 = float(4**3 / 3)
print(exact1)

In [ ]:
# compare the difference with the error bound
integral1[0] - exact1

In [ ]:
# showing extra information about the integral

integral_full = sint.quad(f1, 0, 4, full_output=True)
integral_info = integral_full[2]

for attr in integral_info:
    print(attr, ":", integral_info[attr], "\n")

### Example: $\int_0^\infty e^{-x^2} dx$

In [ ]:
def f2(x):
    return np.exp(-x**2)

integral2 = sint.quad(f2, 0, np.inf)
print(integral2)

In [ ]:
# compare to exact answer
exact2 = float(np.sqrt(np.pi) / 2)
print(exact2)

In [ ]:
# compare the difference with the error bound
integral2[0] - exact2

### Example: $\int_{-1}^{1} \frac{1}{\sqrt{1 - x^2}} dx$

In [ ]:
def f3(x): 
    return 1/np.sqrt(1-x**2)

integral3 = sint.quad(f3, -1, 1)
print(integral3)

In [ ]:
# compare to exact answer
exact3 = float(np.pi)
print(exact3)

In [ ]:
# compare the difference with the error bound
integral3[0] - exact3

### Example: $\int_{0}^{4} \frac{1}{\sqrt[3]{x - 1}} dx$

In [ ]:
## NOTE: use cbrt to correct obtain negative value for cube root of negative number

def f4(x):
    return 1/np.cbrt(x - 1)

In [ ]:
# WRONG: did not specify location of singularity
integral4 = sint.quad(f4, 0, 4)
print(integral4)

In [ ]:
# CORRECT: using the point2 argument
integral4 = sint.quad(f4, 0, 4, points=[1])
print(integral4)

In [ ]:
# compare to exact answer
exact4 = float(1.5 * (3**(2/3) - 1))
print(exact4)

In [ ]:
# compare the difference with the error bound
integral4[0] - exact4

In [ ]:
# using symmetry to calculate an easier equivalent integral
integral4b = sint.quad(f4, 2, 4, points=[1])
print(integral4b)

### Example: volume of paraboloid $z = 4 − 2 x^2 − y^2$ above $z = 0$

In [ ]:
def z(x, y):
    return 4 - 2 * x**2 - y**2

def x_upper(y):
    return np.sqrt(0.5 * (4 -  y**2))

def x_lower(y):
    return -np.sqrt(0.5 * (4 - y**2))

dbl_integral = sint.dblquad(z, -2, 2, x_lower, x_upper)
print(dbl_integral)

In [ ]:
def z_pos(x, y):
    return max(4 - 2 * x**2 - y**2, 0)

dbl_integral_2 = sint.dblquad(z_pos, -2, 2, -np.sqrt(2), np.sqrt(2))
print(dbl_integral_2)

In [ ]:
# compare to exact answer
dbl_exact = float(0.5 * np.pi * 2**4) / np.sqrt(2)
print(dbl_exact)

In [ ]:
# compare the difference with the error bound
dbl_integral[0] - dbl_exact

### Exercise 2

The vertical velocity $v_y$ of a rocket whose initial mass (including fuel) is $m$ is given by:

$$ v_y(t) = - g t + u \ln \left( \frac{m}{m - r t} \right) $$

where $u$ is the expulsion speed of the fuel, $r$ is the rate at which the fuel is consumed, and $g$ = 9.8 m/s$^2$ is the acceleration due to gravity. For $m$ = 15000 kg, $u$ = 2000 m/s and $r$ = 120 kg/s, evaluate the vertical position $y(t) = \int_{0}^{t} v_y(\tau) d\tau$ at sufficiently fine intervals so that you can plot $y(t)$ for $t$ between 0 and 60 s

## Differential equation solving

### Example: $dy/dt = y (1 - y)$; $y(0) = 1/2$

NOTE: $dy/dt = y (1 - y)$ is the logistic equation. It is a first-order, seperable, nonlinear, autonomous differential equation

In [ ]:
# define the RHS of the differential equation as a function
# NOTE that the function needs to take t, y as arguments
# even when there is no dependence on t

def logistic(t, y):
    return y * (1 - y)

In [ ]:
# initial condition: y(t = 0) = 0.5; solve up to t = 5
# solve the differential equation without additional requirements

solved = sint.solve_ivp(logistic, [0, 5], [0.5])
print(solved)

In [ ]:
# solve the differential equation at fine intervals
solved2 = sint.solve_ivp(logistic, [0, 5], [0.5], t_eval=np.linspace(0, 5, 51))
print(solved2)

In [ ]:
# express solution as a function
solved3 = sint.solve_ivp(logistic, [0, 5], [0.5], dense_output=True)
print(solved3)

In [ ]:
# plotting the different results against exact solution
def y_exact(t):
    return 1 / (1 + np.exp(-t))

In [ ]:
t_array = np.linspace(0, 5, 51)

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(t_array, y_exact(t_array), lw=1, label="exact")
ax.plot(solved2.t, solved2.y[0], lw=2, ls="--", label="fine evaluation")
ax.plot(t_array, solved3.sol(t_array)[0], lw=3, ls=":", label="dense output")

ax.legend()

plt.show(fig)

In [ ]:
# obtain the other half of the solution

solved3b = sint.solve_ivp(logistic, [0, -5], [0.5], dense_output=True)
print(solved3b)

In [ ]:
def full_logistic_sol(t):
    return np.where(t < 0, solved3b.sol(t), solved3.sol(t))

In [ ]:
t2_array = np.linspace(-5, 5, 51)

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(t2_array, full_logistic_sol(t2_array)[0], c="tab:green")

plt.show(fig)

### Example: $dy/dt = t^2 - y^2$; $y(0) = 1$

NOTE: $dy/dt = t^2 - y^2$ is a first-order, nonlinear, non-autonomous differential equation

In [ ]:
def Riccati(t, y):
    return t**2 - y**2

In [ ]:
solved = sint.solve_ivp(Riccati, [0, 2], [1], dense_output=True)
print(solved)

In [ ]:
t_array = np.linspace(0, 2, 51)
y_array = solved.sol(t_array)[0]

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(t_array, y_array)

plt.show(fig)

### Example: linear planar first-order ODE

Consider the system 
$$
\begin{align}
dx/dt & = x + 2 y\\
dy/dt & = - x - y
\end{align}
$$
with initial condition $x(0) = 1$ and $y(0) = 0$

In [ ]:
def lin2D(t, y):
    return np.array([y[0] + 2 * y[1], -y[0] -y[1]])

In [ ]:
solved = sint.solve_ivp(lin2D, [0, 10], [1, 0], dense_output=True)
print(solved)

In [ ]:
# solution x(t), y(t) for t of range [0, 10]
t_array = np.linspace(0, 10, 101)
x_array, y_array = solved.sol(t_array)

In [ ]:
# x against time and y against time

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(t_array, x_array, c="tab:orange", ls = "--", label="$x(t)$")
ax.plot(t_array, y_array, c="tab:green", ls = ":", lw=2, label="$y(t)$")

ax.legend()

plt.show(fig)

In [ ]:
# x against y (phase portrait / trajectory)

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(x_array, y_array)

plt.show(fig)

### Example: second-order differential equation

Consider the second order differential equation:

$$ \frac{d^2 y}{dt^2} + 2 \frac{dy}{dt} + 4 y = t \sin(2t)$$

with initial condition $dy/dt(0) = 1$ and $y(0) = 0$

Using the substitution $y_1 = y, y_2 = dy/dt$, we obtain the system:
$$
\begin{align}
dy_1/dt & = y_2 \\
dy_2/dt & = - 2 y_2 - 4 y_1 + t \sin(2t)
\end{align}
$$
and the initial condition becomes $y_1(0) = 0$ and $y_2(0) = 1$

In [ ]:
def sec_order(t, y):
    return np.array([y[1], -4 * y[0] - 2 * y[1] + t * np.sin(2*t)])

In [ ]:
solved = sint.solve_ivp(sec_order, [0, 20], [0, 1], dense_output=True)
print(solved)

In [ ]:
# solution for t in the range [0, 10]
t_array = np.linspace(0, 20, 101)
y_array = solved.sol(t_array)[0]

In [ ]:
# x against time and y against time

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(t_array, y_array)

plt.show(fig)

### Exercise 3

The classic predator-prey model consists of the following system of differential equations (here $x$ is prey and $y$ is predator):
$$\begin{align}
dx/dt & = α x − β x y \\
dy/dt & = − γ y + δ x y
\end{align}$$
With $α$ = 24 yr$^{−1}$, $β$ = 0.4 yr$^{−1}$, $γ$ = 2.0 yr$^{−1}$, $δ$ = 0.002 yr$^{−1}$

**Part 1.** For an initial population of 3000 preys and 50 predators, plot the prey and predator population as a function of time for the range [0 yr, 5 yr]. Plot also the phase portrait for the same time span

**Part 2.** Repeat the previous procedure with the same α, β, γ, and δ, but with:

 A. An initial population of 2000 preys and 50 predators

 B. An initial population of 1000 preys and 50 predators

In a single pair of axes, plot the phase portrait with all 3 sets of initial conditions


### Exercise 4

Consider the following second-order linear differential equation:
$$ t^2 \, d^2y/dt^2 + t \, dy/dt + (t^2 - 1) y = 0$$
Starting with the initial condition $y(1) = 0$ and $dy/dt(1) = 1$, obtain the solution to the differential equation in the $t$ range* (0, 20], and plot the resulting $y$ as a function of $t$

**Note 1:** practically, the open left bound at 0 amounts to a small positive left bound

**Note 2:** you'll likely need to clip the y range of the plot in order to see the feature of the solution in the $t > 1$ region

## Optimization

### Example: minimization of a scalar function

Consider the minimization of the function $f(x) = \log(1 + |x|^{2 + \sin(x)})$

In [ ]:
def f(x):
    return np.log(1 + np.abs(x)**(2 + np.sin(x)))

In [ ]:
x = np.linspace(-10, 10, 101)
y = f(x)

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(x, y)

plt.show(fig)

In [ ]:
# use [-5, 5] as bracket
result = sopt.minimize_scalar(f, [-5, 5])
print(result)

In [ ]:
# extract the value at which minimum is attained
result.x

In [ ]:
# extract the minimum value
result.fun

In [ ]:
# using a bracket that contains no local minimum
sopt.minimize_scalar(f, [-5, -2.5])

In [ ]:
# using a bracket enclose local (but not global) minimum
sopt.minimize_scalar(f, [4, 5])

### Example: minimization of a multivariate function

Consider the minimization of $f(x, y) =  (1 − x)^2 + 100(y − x^2)^2$

In [ ]:
def F(x):
    return (1 - x[0])**2 + 50 * (x[1] - x[0]**2)**2

In [ ]:
x_array = np.linspace(-2, 2, 51)
y_array = np.linspace(-2, 2, 51)

x_grid, y_grid = np.meshgrid(x_array, y_array)
f_grid = F(np.array([x_grid, y_grid]))

fig = plt.figure()
ax = fig.add_subplot()

c = ax.contourf(
    x_array, y_array, f_grid, 
    levels=np.geomspace(1E-5, 1E4, 20),
    norm=mpl.colors.LogNorm(1E-5, 1E3)
)
cb = fig.colorbar(c)
cb.set_ticks(np.geomspace(1E-5, 1E4, 10))

plt.show(fig)

In [ ]:
result = sopt.minimize(F, [-1, -1])
print(result)

### Exercise 5

Maximize the function $f(x_1, x_2, x_3, x_4) = {x_1}^2 {x_2}^3 {x_3}^4 x_4$, where $x_i \geq 0$ for i = 1, 2, 3, 4, subjected to the constraint $x_1 + 2 x_2 + 3 x_3 + x_4 = 100$

Report both the maximum value and the values of $(x1, x2, x3, x4)$ at which the maximum is attained